In [93]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.document_loaders import WebBaseLoader


llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

In [ ]:
class WikipediaSearchToolArgsSchema(BaseModel):

    query: str = Field(..., description="The search query.")


class DuckDuckGoSearchToolArgsSchema(BaseModel):

    query: str = Field(..., description="The search query.")


class DuckDuckGoScraperToolArgsSchema(BaseModel):

    url: str = Field(..., description="The URL to scrape.")


class SaveToFileToolArgsSchema(BaseModel):

    content: str = Field(..., description="The content to save.")
    file_name: str = Field(..., description="The file name to save to.")


class WikipediaSearchTool(BaseTool):

    name = "WikipediaSearch"
    description = """
    Use this tool to search in Wikipedia.
    It takes a query as an argument.
    """
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        wikipedia_api = WikipediaAPIWrapper()
        wikipedia = WikipediaQueryRun(api_wrapper=wikipedia_api)
        return wikipedia.run(query)


class DuckDuckGoSearchTool(BaseTool):

    name = "DuckDuckGoSearch"
    description = """
    Use this tool to search in DuckDuckGo.
    It takes a query as an argument.
    """
    args_schema: Type[DuckDuckGoSearchToolArgsSchema] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query):
        wrapper = DuckDuckGoSearchAPIWrapper(region="wt-wt", time="y")
        search = DuckDuckGoSearchResults(api_wrapper=wrapper)
        return search.run(query)


class DuckDuckGoScraperTool(BaseTool):
    
    name = "DuckDuckGoScraper"
    description = """
    Use this tool when use DuckDuckGoSearchTool.
    Scrape and extract content from a website in DuckDuckGo.
    It takes a URL as an argument.
    """
    args_schema: Type[DuckDuckGoScraperToolArgsSchema] = DuckDuckGoScraperToolArgsSchema

    def _run(self, url):
        loader = WebBaseLoader(url)
        docs = loader.load()
        return docs[0].page_content.replace("\n", "")


class SaveToFileTool(BaseTool):
    
    name = "Save_to_File"
    description = """
    Use this tool when use DuckDuckGoScarperTool. 
    Save DuckDuckGo web research results to a text file.
    It takes content and file_name as arguments.
    """
    args_schema: Type[SaveToFileToolArgsSchema] = SaveToFileToolArgsSchema

    def _run(self, content, file_name):
        if not file_name.endswith(".txt"):
            file_name += ".txt"
        with open(file_name, "w") as f:
            f.write(content)
        return f"Content successfully saved to {file_name}"


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        DuckDuckGoScraperTool(),
        SaveToFileTool(),
    ],
)

In [96]:
prompt = "Research about the XZ backdoor from DuckDuckGo"
agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearch` with `{'query': 'XZ backdoor'}`




TypeError: DDGS.text() got an unexpected keyword argument 'max_results'